Source: https://data.humdata.org/dataset/satellite-detected-water-extents-between-4-11-july-2024-in-bangladesh

In [1]:
import geopandas as gpd
import pandas as pd
import yaml
from rasterio.crs import CRS
from pathlib import Path

In [2]:
yaml_file = '../events/bangladesh_coastal_flood_2024.yml' #'porto_algre.yml'
with open(yaml_file) as f:
    event_dict = yaml.safe_load(f)["event"]
event_dict

{'event_name': 'bangladesh_coastal_flood_2024',
 'bounds': [88.96546265, 21.65505773, 91.33990759, 24.17825791],
 'event_date': '2024-07-08',
 'pre_event_window_days': 100,
 'post_event_window_days': 60,
 'rtc_track_numbers': [150, 114],
 'mgrs_tiles': ['45QYE', '45QYF', '45QZE', '45QZF'],
 'dist_hls_confirmed_change_min_days': 30,
 'source_id': 'UNOSAT via humanitarian data exchange',
 'links': ['https://data.humdata.org/dataset/satellite-detected-water-extents-between-4-11-july-2024-in-bangladesh',
  'https://reliefweb.int/disaster/fl-2024-000088-bgd']}

In [47]:
df_flood = gpd.read_file('../external_validation_data/ST1_20240704_20240711_FloodExtent_Bangladesh')
df_flood = df_flood.to_crs(4326)
df_flood.head()

,FID_bgd_ad,Shape_Leng,ADM3_EN,ADM3_PCODE,ADM2_EN,ADM2_PCODE,ADM1_EN,ADM1_PCODE,ADM0_EN,ADM0_PCODE,FL_Area,geometry
0,0,1.171879,Abhaynagar,BD404104,Jessore,BD4041,Khulna,BD40,Bangladesh,BD,14.676618,"MULTIPOLYGON (((89.51577 22.92842, 89.51568 22..."
1,4,0.687378,Agailjhara,BD100602,Barisal,BD1006,Barisal,BD10,Bangladesh,BD,2.021449,"MULTIPOLYGON (((90.18070 22.90734, 90.18061 22..."
2,8,1.092474,Alamdanga,BD401807,Chuadanga,BD4018,Khulna,BD40,Bangladesh,BD,0.243772,"MULTIPOLYGON (((89.00759 23.62812, 89.00759 23..."
3,9,0.777981,Alfadanga,BD302903,Faridpur,BD3029,Dhaka,BD30,Bangladesh,BD,1.220706,"MULTIPOLYGON (((89.68752 23.21052, 89.68752 23..."
4,11,1.846319,Amtali,BD100409,Barguna,BD1004,Barisal,BD10,Bangladesh,BD,51.657308,"MULTIPOLYGON (((90.07247 21.86330, 90.07202 21..."


In [48]:
# df_flood_coastal = df_flood.cx[89.2:91, 21.5: 24].reset_index(drop=True)
# df_flood_coastal.explore()

In [49]:
df_flood_coastal.total_bounds

array([88.96546265, 21.65505773, 91.33990759, 24.17825791])

In [50]:
n = df_flood_coastal.shape[0]
df_event_f = gpd.GeoDataFrame({'name': [event_dict['event_name'] for k in range(n)],
                               'date': [pd.to_datetime(event_dict['event_date']) for k in range(n)],
                               'source_id': [f'{event_dict["source_id"]}' for k in range(n)]},
                              geometry=df_flood_coastal.geometry.tolist(),
                              crs=CRS.from_epsg(4326))
df_event_f.head()

,name,date,source_id,geometry
0,bangladesh_coastal_flood_2024,2024-07-08,UNOSAT via https://data.humdata.org,"MULTIPOLYGON (((89.51577 22.92842, 89.51568 22..."
1,bangladesh_coastal_flood_2024,2024-07-08,UNOSAT via https://data.humdata.org,"MULTIPOLYGON (((90.18070 22.90734, 90.18061 22..."
2,bangladesh_coastal_flood_2024,2024-07-08,UNOSAT via https://data.humdata.org,"MULTIPOLYGON (((89.68752 23.21052, 89.68752 23..."
3,bangladesh_coastal_flood_2024,2024-07-08,UNOSAT via https://data.humdata.org,"MULTIPOLYGON (((90.07247 21.86330, 90.07202 21..."
4,bangladesh_coastal_flood_2024,2024-07-08,UNOSAT via https://data.humdata.org,"MULTIPOLYGON (((89.24744 22.35108, 89.24717 22..."


In [51]:
EXTERNAL_VAL_DATA_DIR = Path('../external_validation_data_db/')
EXTERNAL_VAL_DATA_DIR.mkdir(exist_ok=True, parents=True)

In [52]:
df_event_f.to_parquet(EXTERNAL_VAL_DATA_DIR / f'{event_dict["event_name"]}.parquet')

In [37]:
#df_event_f.to_file(EXTERNAL_VAL_DATA_DIR / f'{event_dict["event_name"]}.geojson', driver='GeoJSON')